Author: Hiranya Jayakody. April 2020.

Code developed for Smart Robotics Viticulture Group, UNSW, Sydney.

Neural Network based on Matterport implementation of Mask-RCNN at https://github.com/matterport/Mask_RCNN

### PART 1.1: Install Mask-RCNN repo from Matterport

In [17]:
!git clone https://github.com/matterport/Mask_RCNN.git
!pip install -r 'Mask_RCNN/requirements.txt'
!cd Mask_RCNN ; python setup.py install

D:\Projects\General\Python\stomata_counter


In [1]:
!pip show mask-rcnn

Name: mask-rcnn
Version: 2.1
Summary: Mask R-CNN for object detection and instance segmentation
Home-page: https://github.com/matterport/Mask_RCNN
Author: Matterport
Author-email: waleed.abdulla@gmail.com
License: MIT
Location: d:\projects\web\back-end\stomata-counter-api\env\lib\site-packages\mask_rcnn-2.1-py3.7.egg
Requires: 
Required-by: 


In [0]:
!pip install tensorflow==1.5.1

In [0]:
!pip install keras==2.1.5

In [0]:
!pip install opencv-python==3.4.9.31

### PART 1.2: Set-up workspace

Option 1: Local

In [1]:
import os

CWD = os.getcwd()
STOMATA_WEIGHTS_PATH = os.path.join(CWD,'weights/2020_mask_rcnn_stomata_51.h5') 
WEIGHT_FILE_NAME = 'stomata'
CLASS_NAME = 'stomata'
DATASET_DIR = os.path.join(CWD,'images/')
INPUT_IMG_DIR = os.path.join(DATASET_DIR,'test/') 
OUTPUT_IMG_DIR = os.path.join(DATASET_DIR,'results/')

Option 2: Google Drive

In [0]:
#mount your google drive if necessary
from google.colab import drive
drive.mount('/content/drive')

Option 3: AWS S3

In [2]:
import cv2
import numpy as np
import boto3

# defining s3 bucket object
s3 = boto3.client("s3")
bucket_name = "ainz11-test"

In [14]:
# list all file in bucket
response = s3.list_objects_v2(
    Bucket=bucket_name
)
for obj in response.get('Contents', None):
    print(obj.get('Key', None))

dog.jpg
meme.jpg
stomata1.jpg


In [3]:
# import time
# start_time = time.time()
# print("--- %s seconds ---" % (time.time() - start_time))

# fetching object from bucket
file_obj = s3.get_object(Bucket=bucket_name, Key="stomata1.jpg")
# reading the file content in bytes
file_content = file_obj["Body"].read()

# creating 1D array from bytes data range between[0,255]
# np_array = np.frombuffer(file_content, np.uint8)
# decoding array
# image_np = cv2.imdecode(np_array, cv2.IMREAD_COLOR)
# -> in one line:
image_np = cv2.imdecode(np.asarray(bytearray(file_content)), cv2.IMREAD_COLOR)

# converting image from RGB to Grayscale
# gray = cv2.cvtColor(image_np, cv2.COLOR_BGR2GRAY)

# # saving image to tmp (writable) directory
# cv2.imwrite("gray_obj.jpg", gray)


In [24]:
# Using boto3 "Resource" --> slower
# s3 = boto3.resource('s3', region_name='us-east-2')
# bucket = s3.Bucket(bucket_name)
# img = bucket.Object("stomata1.jpg").get().get('Body').read()
# image_np = cv2.imdecode(np.asarray(bytearray(img)), cv2.IMREAD_COLOR)

--- 1.6653015613555908 seconds ---


In [8]:
# uploading converted image to S3 bucket
# s3.put_object(Bucket=bucket_name, Key="grayscale.jpg", Body=open("gray_obj.jpg", "rb").read())

{'ResponseMetadata': {'RequestId': 'FB30DA2FF7B05F70',
  'HostId': '25LUR5rYjDIYB1Uu/f7auFYI1Jbxr9KHixot2GNaAscIICf9ft7r3M+EQylrqp6eC9GmucwC7ZQ=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '25LUR5rYjDIYB1Uu/f7auFYI1Jbxr9KHixot2GNaAscIICf9ft7r3M+EQylrqp6eC9GmucwC7ZQ=',
   'x-amz-request-id': 'FB30DA2FF7B05F70',
   'date': 'Mon, 19 Oct 2020 06:13:10 GMT',
   'etag': '"693f053ec58adb36f7e7eef985406441"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"693f053ec58adb36f7e7eef985406441"'}

### PART 2: Set-up Mask-RCNN for inference

In [4]:
# restart runtime if 'No module named 'mrcnn'' error occurs

import cv2
import glob
import sys
import json
import datetime
import numpy as np
import skimage.draw
import imutils
import imgaug
import statistics as st
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

from mrcnn.config import Config
from mrcnn import visualize
from mrcnn import model as modellib, utils
from matplotlib import pyplot as plt

Using TensorFlow backend.


In [5]:
#create config for inference
class InferenceConfig(Config):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    NAME = CLASS_NAME #provide a suitable name
    NUM_CLASSES = 1+1 #background+number of classes
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    RPN_ANCHOR_SCALES = (12,24,48,96,192) #anchor box scales for the application
    DETECTION_MIN_CONFIDENCE = 0.6 #set min confidence threshold
    DETECTION_MAX_INSTANCES = 500
    POST_NMS_ROIS_INFERENCE = 10000
    IMAGE_MAX_DIM = 1024
    IMAGE_MIN_DIM = 800
    MEAN_PIXEL = np.array([133.774, 133.774, 133.774]) #DEFAULT VALUES FOR STOMATA MODEL. Change as necessary. #matterport takes the input as RGB

In [6]:
#create inference object
inference_config = InferenceConfig()
inference_config.display()

# Load the model in inference mode
model = modellib.MaskRCNN(mode="inference", config=inference_config, model_dir=CWD)
model_path = STOMATA_WEIGHTS_PATH #os.path.join(CWD,'mask_rcnn_stomata.h5')

print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        500
DETECTION_MIN_CONFIDENCE       0.6
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

### PART 3: Apply model to identify stomata

3.1 Function Definitions

In [7]:
# Statistical filter

def stomata_filter(r_pd_):
    #stomata filer: This code filters out stomata like shapes which are of wrong size, using confidence measures.
    high_scores = r_pd_["scores"][r_pd_["scores"]>0.90]
            
    if len(high_scores) > 0:
        percentile_thres = np.min(high_scores)
    else:
        percentile_thres = np.percentile(r_pd_["scores"], 95) #st.median(r_pd_["scores"])
    
    high_conf_areas = r_pd_["areas"][r_pd_["scores"]>=percentile_thres] #conf_threshold
    high_conf_scores = r_pd_["scores"][r_pd_["scores"]>=percentile_thres]
    
    high_conf_avg_area = st.mean(high_conf_areas)
    
    above_avg = high_conf_areas[high_conf_areas>=high_conf_avg_area]
    below_avg = high_conf_areas[high_conf_areas<high_conf_avg_area]
    
    above_avg_conf = high_conf_scores[high_conf_areas>=high_conf_avg_area]
    below_avg_conf = high_conf_scores[high_conf_areas<high_conf_avg_area]
    
    #based on data length
    if len(above_avg) >= len(below_avg):
        optimal_area = np.percentile(above_avg, 50) #(st.mean(above_avg)+np.max(above_avg))/2.0 #can we use percentie 
        st_size = 'LARGE'
    else:
        #smaller elements may not be stomata, so check for their overall confidence with respect to the confidence of larger areas
        if np.mean(above_avg_conf) >= np.mean(below_avg_conf) and len(above_avg) > 1:
            optimal_area = np.percentile(above_avg, 50)
            st_size = 'LARGE'
        elif len(above_avg) <=1 and np.max(above_avg_conf) > 0.985:
            optimal_area = np.percentile(above_avg, 50)
            st_size = 'LARGE'
        else:
            optimal_area = np.percentile(below_avg, 75)
            st_size = 'SMALL'
       
    if st_size == 'LARGE':
        indices_ = r_pd_["scores"][np.logical_and(r_pd_["areas"]> (optimal_area*0.55),r_pd_["areas"]<1.5*optimal_area )].index.values.astype(int)
    else:
        indices_ = r_pd_["scores"][np.logical_and(r_pd_["areas"]> (optimal_area*0.65),r_pd_["areas"]<1.5*optimal_area )].index.values.astype(int)

    return indices_
    



In [8]:
# Other supporting functions

def get_filename(string_):
    #get image filename    
    start = string_.find('test/')+5
    end = string_.find('.jpg',start)
    filename = string_[start:end]
    return filename

def hisEqulColor(img):
    #contrast limited histogram equalisation
    ycrcb=cv2.cvtColor(img,cv2.COLOR_BGR2YCR_CB)
    channels=cv2.split(ycrcb)
    clahe = cv2.createCLAHE(clipLimit=1.0, tileGridSize=(25,25))
    channels[0] = clahe.apply(channels[0])
    cv2.merge(channels,ycrcb)
    cv2.cvtColor(ycrcb,cv2.COLOR_YCR_CB2BGR,img)
    return img

def sharpenColor(img):
    #sharpen image
    kernel_sharpening = np.array([[-1,-1,-1], 
                              [-1, 9,-1],
                              [-1,-1,-1]])
    img = cv2.filter2D(img, -1, kernel_sharpening)
    return img




### PART 3A: Test on a single image ###

Please Refer to Part 3B and remove the loop for the folder.

### PART 3B: Test on an image folder

In [9]:
DATA_PATH = os.path.join(INPUT_IMG_DIR,'*jpg')
# files = glob.glob(DATA_PATH)
files = [image_np]

stomata_data = pd.DataFrame(columns=['filename','num_stomata','scores','areas'])

print(cv2. __version__)

3.4.9


In [12]:
counter = 0
# for img in files:
#     filename = get_filename(img)
#     print(filename)
#     image = cv2.imread(img)
img = files[0]
filename = "test"
image = imutils.resize(img,width=1024)
image_original = image
image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB) #opnecv uses BGR convention


#call histogram equalize function (optional)
image = hisEqulColor(image)
#image = sharpenColor(image)

#convert to grayscale and save as a three channel jpeg then read it back and convert
image_gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
cv2.imwrite('current_image.jpg', image_gray)

image_new = cv2.imread('current_image.jpg')
image = cv2.cvtColor(image_new,cv2.COLOR_BGR2RGB) #opnecv uses BGR convention
image = imutils.resize(image,width=1024)

#run the image through the model
print("making predictions with Mask R-CNN...")
r = model.detect([image], verbose=1)[0]

#create dataframe for ease of use
r_pd = pd.DataFrame(columns=['class_id','scores', 'areas'])

#create array to store areas
num_stomata = len(r["scores"])

r_pd["class_ids"] = r["class_ids"]
r_pd["scores"] = r["scores"]

#retrieve area values from X,Y coordinates
for i in range(0, len(r_pd["scores"])): 
    (startY,startX,endY,endX) = r["rois"][i]
    r_pd["areas"][i] = abs(startY-endY)*abs(startX-endX)


#see how many stomata are on the image
#1. If there are more than 2 stomata, do the following.
#2. get the median score for confidence
#3. get the average area for median and above
#4. reject areas 90% or less than the average median area

if num_stomata == 0:
    print("no stomata detected")
    stomata_data.loc[counter] = [filename,num_stomata,0]
    counter +=1
    cv2.imwrite(OUTPUT_IMG_DIR+filename+'_000'+'.jpg', image)
    sys.exit()

if num_stomata >= 2:

    indices = stomata_filter(r_pd)
    #indices = r_pd["scores"][:].index.values.astype(int) #this ignores the statistical filter

else: 
    indices = [0]

print(indices)

# loop over of the detected object's bounding boxes and masks
areas = []
for i in range(0, len(indices)):
    classID = r["class_ids"][indices[i]]
    mask = r["masks"][:, :, indices[i]]
    color = [0,0,0]
    areas.append(np.sum(mask == True))

    #uncomment to visualize the pixel-wise mask of the object
    #image = visualize.apply_mask(image, mask, color, alpha=0.5)

    #visualize contours
    mask[mask ==True] = 1
    mask[mask == False] = 0
    mask = mask.astype(np.uint8)
    mask,contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    cv2.drawContours(image_original,contours, 0, [0,255,0], 4)

image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

for i in range(0,len(indices)):
    (startY,startX,endY,endX) = r["rois"][indices[i]]
    classID = r["class_ids"][indices[i]]
    label = classID
    score = r["scores"][indices[i]]
    color = [255,0,0]

    #uncomment to draw bounding box around stomata
    #cv2.rectangle(image_original,(startX,startY),(endX,endY),color,2)

    #uncomment to print confidence value
    #text = "{}: {:.3f}".format(label,score)
    #y = startY - 10 if startY - 10 > 10 else startY + 10
    #cv2.putText(image_original,text,(startX,y),cv2.FONT_HERSHEY_SIMPLEX,0.8,color,2)

id_str= str(len(indices))
stomata_data.loc[counter] = [filename,len(indices),r["scores"][indices],areas]
counter +=1
cv2.imwrite(OUTPUT_IMG_DIR+filename+'_'+id_str.zfill(3)+'.jpg', image_original)
    

stomata_data.to_csv(OUTPUT_IMG_DIR+'results.csv',encoding='utf-8', index=False)
    
    

making predictions with Mask R-CNN...
Processing 1 images
image                    shape: (771, 1024, 3)        min:   23.00000  max:  224.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -133.77400  max:   90.22600  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.13271  max:    1.07015  float32


d:\projects\web\back-end\stomata-counter-api\env\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


[0 1 2 3]
1
1
1
1


In [13]:
print(r_pd["class_ids"])

0    1
1    1
2    1
3    1
4    1
Name: class_ids, dtype: int32
